In [1]:
import os
import ast
import json
import glob
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
#import cudf
import joblib

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn import preprocessing
from tqdm import tqdm

In [2]:
train_w1 = dict()
label_w1 = dict()
# train_w2 = dict()
# train_w3 = dict()
# train_w4 = dict()

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/train_w0_part*.parquet"):
   train_w1[file] = pd.read_parquet(file)
train_w1 = pd.concat(train_w1.values())

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/label_w0_part*.parquet"):
   label_w1[file] = pd.read_parquet(file)
label_w1 = pd.concat(label_w1.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w1_part*.parquet"):
#    train_w2[file] = pd.read_parquet(file)
# train_w2 = pd.concat(train_w2.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w2_part*.parquet"):
#    train_w3[file] = pd.read_parquet(file)
# train_w3 = pd.concat(train_w3.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w3_part*.parquet"):
#    train_w4[file] = pd.read_parquet(file)
# train_w4 = pd.concat(train_w4.values())


#max aid unique 486

In [3]:
#label encoding aids

def get_le():
    temp_all_data = pd.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')


    le = preprocessing.LabelEncoder()
    le.fit(temp_all_data['aid'])
    del temp_all_data
    return le


def label_encoding(df,le):


    df['aid'] = le.transform(df['aid'])

    #set start = 2 to reserve 0 for mask and 1 for pad
    df['aid+2'] = df['aid']+2
    df['aid'] = df['aid+2']

    df = df.drop(['aid+2'], axis = 1)
    joblib.dump(le, 'label_encoder.joblib')
    return df

In [4]:
le = get_le()
train_w1 = label_encoding(train_w1, le)

In [5]:
def get_merged(df, label_df):

    df = pd.DataFrame(df.groupby('session')['aid'].unique().agg(list))
    label_df = pd.DataFrame(label_df.groupby('session')['aid'].unique().agg(list))
    
    df.rename(columns = {'aid': 'input'}, inplace = True)
    label_df.rename(columns = {'aid': 'label'}, inplace = True)

    df = df.reset_index()
    label_df = label_df.reset_index()

    df = pd.merge(df, label_df, on = "session", how = "inner")  
    
    return df

In [6]:
train_w1 = get_merged(train_w1, label_w1)
del label_w1

In [ ]:
INPUT_LENGTH = 486
OUTPUT_LENGTH = 433

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sessions, input_length = INPUT_LENGTH, output_length = OUTPUT_LENGTH):
        self.sessions = sessions
        self.input_length = input_length
        self.output_length = output_length
        self.history_size = history_size

    def __len__(self):
        return len(self.sessions)
    
    def pad_items(self, session):
        if len(session)< length:
            session = session + list(length - len(session) * [0])
        else: session = session[:length]
        return session
      
    def __getitem__(self, idx):
        input_tokens = sessions.iloc[idx, sessions.columns.get_loc("input")]
        input_tokens = pad_items(input, self.input_length) 
        
        target = sessions.iloc[idx, sessions.columns.get_loc("label")]
        target = pad_items(target, self.output_length) 
        
        input_tokens = torch.tensor(input_tokens, dtype=torch.long)
        target = torch.tensor(target, dtype=torch.long)
        
        return input_tokens, target
                                     

In [ ]:
class Recommender(pl.LightningModule):
    def __init__(
        self,
        vocab_size,
        channels=128,
        cap=0,
        mask=1,
        dropout=0.4,
        lr=1e-4,
    ):
        super().__init__()

        self.cap = cap
        self.mask = mask

        self.lr = lr
        self.dropout = dropout
        self.vocab_size = vocab_size

        self.item_embeddings = torch.nn.Embedding(
            self.vocab_size, embedding_dim=channels
        )

        self.input_pos_embedding = torch.nn.Embedding(512, embedding_dim=channels)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels, nhead=4, dropout=self.dropout
        )

        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=6)

        self.linear_out = Linear(channels, self.vocab_size)

        self.do = nn.Dropout(p=self.dropout)

    def encode_src(self, src_items):
        src_items = self.item_embeddings(src_items)

        batch_size, in_sequence_len = src_items.size(0), src_items.size(1)
        pos_encoder = (
            torch.arange(0, in_sequence_len, device=src_items.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )
        pos_encoder = self.input_pos_embedding(pos_encoder)

        src_items += pos_encoder

        src = src_items.permute(1, 0, 2)

        src = self.encoder(src)

        return src.permute(1, 0, 2)

    def forward(self, src_items):

        src = self.encode_src(src_items)

        out = self.linear_out(src)

        return out

    def training_step(self, batch, batch_idx):
        src_items, y_true = batch

        y_pred = self(src_items)

        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)

        src_items = src_items.view(-1)
        mask = src_items == self.mask

        loss = masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)

        self.log("train_loss", loss)
        self.log("train_accuracy", accuracy)

        return loss

    def validation_step(self, batch, batch_idx):
        src_items, y_true = batch

        y_pred = self(src_items)

        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)

        src_items = src_items.view(-1)
        mask = src_items == self.mask

        loss = masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)

        self.log("valid_loss", loss)
        self.log("valid_accuracy", accuracy)

        return loss

    def test_step(self, batch, batch_idx):
        src_items, y_true = batch

        y_pred = self(src_items)

        y_pred = y_pred.view(-1, y_pred.size(2))
        y_true = y_true.view(-1)

        src_items = src_items.view(-1)
        mask = src_items == self.mask

        loss = masked_ce(y_pred=y_pred, y_true=y_true, mask=mask)
        accuracy = masked_accuracy(y_pred=y_pred, y_true=y_true, mask=mask)

        self.log("test_loss", loss)
        self.log("test_accuracy", accuracy)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=10, factor=0.1
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "valid_loss",
        }

In [ ]:
train_w1['input'] = train_w1.apply(lambda x: x['aid'] + 
                                      list((TRAIN_HISTORY - len(x['aid'])) * [0]), 
                                      axis = 1)

In [ ]:
def get_input_target(df, CHUNK = 100000):
    
    #train_w_dict = {}
    it = len(train_w1)//CHUNK +1

    for i in tqdm(range(it)):
        if i < it-1: temp = df.iloc[i*CHUNK: (i+1)*CHUNK,:].copy()
        else: temp = df.iloc[i*CHUNK:,:].copy()
        temp['input'] = temp.apply(lambda x: x['aid'] + 
                                      list((TRAIN_HISTORY - len(x['aid'])) * [0]), 
                                      axis = 1)
        temp['target'] = temp.apply(lambda x: x['label'] + 
                                      list((PREDICTION_HISTORY - len(x['aid'])) * [0]), 
                                      axis = 1)
        
        temp.drop(['aid', 'label'], axis = 1, inplace = True)
        temp.to_parquet(f'/kaggle/working/input_target_part_{i}.parquet')
        
        #train_w_dict[i] = temp
        
        del temp

#     for file in glob.glob("/kaggle/working/input_target_part_*.parquet"):
#        train_w_dict[file] = pd.read_parquet(file)
    #df = pd.concat(train_w_dict.values())    
    #return df
    

In [ ]:
#train_w1 = get_input_target(train_w1)
get_input_target(train_w1)

In [ ]:
del train_w1
gc.collect()

In [ ]:
train_w1 = pd.DataFrame()

In [ ]:
for file in glob.glob("/kaggle/working/input_target_part_*.parquet"):
    temp = pd.read_parquet(file)
    train_w1 = pd.concat([train_w1, temp], ignore_index = True)  
    del temp
    gc.collect()